# Paths and pavement data from NISMOD-DB

Original source OS MasterMap topography layer, Crown copyright.

In [ ]:
import configparser
import glob
import json
import os

import pandas
import geopandas
import requests
import shapely.wkt
from tqdm.notebook import tqdm

In [ ]:
def get_auth():    
    # Read connection details
    if 'NISMOD_API_USER' in os.environ and 'NISMOD_API_PASSWORD' in os.environ:
        username = os.environ['NISMOD_API_USER']
        password = os.environ['NISMOD_API_PASSWORD']
    else:
        parser = configparser.ConfigParser()
        parser.read('dbconfig.ini')
        username = parser['nismod-api']['user']
        password = parser['nismod-api']['password']

    return (username, password)

In [ ]:
CACHE_PATH = os.path.join('.', 'db-data')
AUTH = get_auth()

In [ ]:
r = requests.get(
    'https://www.nismod.ac.uk/api/data/mastermap/area_classes',
    auth=AUTH
)
feature_classes = pandas.DataFrame(r.json())
feature_classes.to_csv('feature_classes.csv')
feature_classes.head()

## Extract classification codes of interest, from manual review of feature classes.

With reference to http://www.ordnancesurvey.co.uk/documents/os-mastermap-real-world-object-catalogue.pdf

In [ ]:
feature_codes = [
    10123,
    10172,
    10183,
]
feature_classes_of_interest = feature_classes[feature_classes.feature_code.isin(feature_codes)]
feature_classes_of_interest

In [ ]:
def get_lads(force=False):    
    try:
        os.mkdir(os.path.join(CACHE_PATH))
    except FileExistsError:
        pass
    fname = os.path.join(CACHE_PATH, "lads.json")

    if not os.path.exists(fname) or force:
        r = requests.get(
            'https://www.nismod.ac.uk/api/data/boundaries/lads',
            auth=auth,
            params={
                'lad_codes': 'all',
            },
            stream=True
        )
        with open(fname, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)

In [ ]:
get_lads()

In [ ]:
def read_to_gdf(fname):
    with open(fname) as fh:
        df = json.load(fh)
        df = geopandas.GeoDataFrame(df)
        df.geometry = df.geom.apply(lambda wkt: shapely.wkt.loads(wkt))
        df.drop("geom", axis=1, inplace=True)
    return df

In [ ]:
lads = read_to_gdf(os.path.join(CACHE_PATH, "lads.json"))
lads.head()

In [ ]:
lads[['lad_code']].to_csv('lads.txt', index=False)

In [ ]:
lads.to_file(os.path.join(CACHE_PATH, "lads.gpkg"), driver="GPKG")

In [ ]:
def get_features(area_code, area_type, classes, force=False):
    try:
        os.mkdir(os.path.join(CACHE_PATH))
    except FileExistsError:
        pass
    fname = os.path.join(CACHE_PATH, "features_{}.json".format(area_code))

    if not os.path.exists(fname) or force:
        r = requests.get(
            'https://www.nismod.ac.uk/api/data/mastermap/areas',
            auth=auth,
            params={
                'scale': area_type,
                'area_codes': area_code,
                'classification_codes': join(classes)
            },
            stream=True
        )
        with open(fname, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)

In [ ]:
def join(list_, delim=","):
    return delim.join(str(element) for element in list_)

In [ ]:
%time get_features('E00003069', 'oa', feature_codes)

In [ ]:
%time get_features('E06000001', 'lad', feature_codes)

In [ ]:
for lad_code in tqdm(lads.lad_code):
    get_features(lad_code, 'lad', feature_codes)

In [ ]:
def read_filter_write(lad_code):
    # read
    try:
        fname = glob.glob(os.path.join(CACHE_PATH, f"features_{lad_code}*"))[0]
    except Exception as err:
        print(lad_code, err)
        return
    
    output_fname = os.path.join(CACHE_PATH, f'{lad_code}.gpkg')
    
    if os.path.exists(output_fname):
        print(lad_code, "Skipped as exists already")
        return
    
    with open(fname) as fh:
        try:
            df = pandas.DataFrame(json.load(fh))
        except Exception as err:
            print(lad_code, err)
            return
        
    if not len(df):
        print(lad_code, "No data")
        return
    
    # treat columns for easier filtering (data are lists, mostly single-element)
    df.descriptive_group = df.descriptive_group.apply(join)
    df.theme = df.theme.apply(join)
    
    # filter
    df = df[(df.descriptive_group == 'Roadside') & (df.make == 'Manmade')]
    
    # convert to geodataframe
    df['geometry'] = df.geom.apply(shapely.wkt.loads)
    df.drop("geom", axis=1, inplace=True)        
    df = geopandas.GeoDataFrame(df)
    
    # write
    df.to_file(output_fname, driver='GPKG')

In [ ]:
for lad_code in tqdm(lads.lad_code):
    read_filter_write(lad_code)